In [7]:
from ansys.mapdl.core import launch_mapdl

def launch_mapdl_on_available_port(starting_port=50052, max_attempts=10):
    for i in range(max_attempts):
        port = starting_port + i
        try:
            mapdl = launch_mapdl(port=port)
            print(f"MAPDL launched successfully on port {port}")
            return mapdl
        except Exception as e:
            print(f"Failed to launch MAPDL on port {port}: {e}")
    raise RuntimeError("Could not launch MAPDL on any available port")

# Use a função para iniciar uma instância do MAPDL
mapdl = launch_mapdl_on_available_port()

MAPDL launched successfully on port 50052


In [26]:
mapdl.finish()
mapdl.clear('nostart')

mapdl.prep7()
mapdl.title('Análise de Viga e Casca com Concreto')

# Definir o tipo de elemento (BEAM188) e suas propriedades
mapdl.et(1, 'BEAM188')
mapdl.keyopt(1, 3, 3)  # Sugerido para BEAM188

# Propriedades do material para BEAM188 (Concreto)
mapdl.mp('EX', 1, 3e6)  # Módulo de elasticidade do concreto em N/cm²
mapdl.mp('PRXY', 1, 0.2)  # Coeficiente de Poisson do concreto
mapdl.mp('DENS', 1, 2.4e-3)  # Densidade do concreto em kg/cm³

# Propriedades da seção da viga em centímetros
mapdl.sectype(1, 'BEAM', 'CSOLID')
mapdl.secoffset('CENT')
mapdl.secdata(0.50)

# Definir os pontos iniciais e finais das estacas
x_iniciais = [0.0, 2.0, 4.0, 6.0, 3.0, 4.0, 0.0, 2.0, 4.0, 6.0]
y_iniciais = [3.0, 3.0, 3.0, 3.0, 0.0, 0.0, -3.0, -3.0, -3.0, -3.0]
z_iniciais = [0.0] * 10

x_finais = [0.0, 2.0, 4.0, 6.0, 3.0, 4.0, 0.0, 2.0, 4.0, 6.0]
y_finais = [3.0, 3.0, 3.0, 3.0, 0.0, 0.0, -3.0, -3.0, -3.0, -3.0]
z_finais = [-20.0] * 10

num_intermediate_nodes = 5
node_id = 1

# Loop para adicionar nós
for i in range(10):
    x_inicial = x_iniciais[i]
    y_inicial = y_iniciais[i]
    z_inicial = z_iniciais[i]

    x_final = x_finais[i]
    y_final = y_finais[i]
    z_final = z_finais[i]

    # Adicionar o nó inicial
    mapdl.n(node_id, x_inicial, y_inicial, z_inicial)

    # Adicionar o nó final
    node_id_final = node_id + num_intermediate_nodes + 1
    mapdl.n(node_id_final, x_final, y_final, z_final)
    mapdl.d(node_id_final, 'ALL', 1)  # Trava os nós finais

    # Preencher nós intermediários
    mapdl.fill(node_id, node_id_final, num_intermediate_nodes)

    # Atualizar o node_id para a próxima iteração
    node_id = node_id_final + 1

n_elemento = 1
# Geração de elementos
for i in range(10):
    for j in range(1, num_intermediate_nodes + 2):
        N_1 = j + (num_intermediate_nodes + 2) * i
        N_2 = N_1 + 1
        mapdl.en(n_elemento, N_1, N_2)
        n_elemento += 1

# Definição do tipo de elemento (SHELL181)
mapdl.et(2, 'SHELL181')

# Definição da seção de casca
mapdl.sectype(2, 'SHELL')
mapdl.secdata(0.1)

# Propriedades do material para SHELL181
mapdl.mp('EX', 2, 2e5)  # Módulo de elasticidade
mapdl.mp('PRXY', 2, 0.3)  # Coeficiente de Poisson
mapdl.mp('DENS', 2, 7.85e-3)  # Densidade em kg/cm³ (ajuste conforme necessário)

# Criação do retângulo e malha da casca
mapdl.rectng(0, 6, -3, 3)  # Cria um retângulo
mapdl.esize(0.1)  # A densidade de elementos
mapdl.aatt(2, 0, 2, 0, 2)
mapdl.amesh('ALL')  # Atribuir propriedades de seção, material e tipo de elemento para os elementos selecionados

# Merge pontos
mapdl.nsel('S', 'LOC', 'Z', 0, 1e5)
mapdl.nummrg('NODE', 1e-5)
mapdl.nsel('ALL')

# Definir condições de carregamento e fronteira
# (Ajustar conforme necessário para seu caso específico)
mapdl.nsel('S', 'LOC', 'X', 0)
mapdl.d('ALL', 'UX', 0)
mapdl.d('ALL', 'UY', 0)
mapdl.d('ALL', 'UZ', 0)
mapdl.nsel('ALL')

# Saída dos nós e elementos
mapdl.allsel('ALL')  # Seleciona tudo
mapdl.run('/STAT')  # Exibe o status da análise
nodes = mapdl.nlist('ALL', 'XYZ')  # Exibe a lista dos nós com coordenadas
elements = mapdl.elist('ALL')  # Exibe a lista dos elementos e seus nós

#print(nodes)
#print(elements)
# Resolver a análise
mapdl.run('/SOLU')
mapdl.solve()
mapdl.finish()

# Pós-processamento
mapdl.post1()
reactions = mapdl.prrsol()  # Lista as reações em arquivo de texto
# Imprimir as informações das reações
print(reactions)

# Código para a obtenção dos esforços normais
mapdl.avprin(0)
mapdl.etable('Axial1', 'SMISC',1 )
mapdl.avprin(0)
mapdl.etable('Axial2', 'SMISC', 12)

# Imprimir as informações
axial1_data = mapdl.get_value('ETABLE', 'Axial1')
axial2_data = mapdl.get_value('ETABLE', 'Axial2')

#print(axial1_data)
#print(axial2_data)

# Obter e imprimir as resultantes axiais para elementos de 1 a 60
axial_results = []
for elem_id in range(1, 61):
    axial1 = mapdl.get_value('ETABLE', 'Axial1', elem=elem_id)
    axial2 = mapdl.get_value('ETABLE', 'Axial2', elem=elem_id)
    axial_results.append((elem_id, axial1, axial2))

# Imprimir os resultados axiais
for result in axial_results:
    print(f'Elemento {result[0]}: Axial1 = {result[1]}, Axial2 = {result[2]}')

# Obter rotações
#mapdl.etable('Rot1', 'ROT', '')

# Obter deslocamentos
#mapdl.etable('Des1', 'U', '')

# Print outputs
#print(nodes)
#print(elements)

PRINT REACTION SOLUTIONS PER NODE

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R1          24.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   23:25:44  JUL 08, 2024 CP=      9.312

 Análise de Viga e Casca com Concreto                                         


 
  ***** POST1 TOTAL REACTION SOLUTION LISTING *****                            
 
  LOAD STEP=     1  SUBSTEP=     1                                             
   TIME=    1.0000      LOAD CASE=   0                                         
 
  THE FOLLOWING X,Y,Z SOLUTIONS ARE IN THE GLOBAL COORDINATE SYSTEM            
 
    NODE       FX           FY           FZ           MX           MY           MZ    
       1  -618.55       137.70      -15.213     
       2  -162.00       160.96      0.10677E-010
       3   742.42      -742.18     -0.44718E-010
       4  -3272.1       3272.0      0.19617E-009
       5   14393.      -14393.     -0.86266E-009
       6  -63302.   

In [27]:
#Sair da port do mapdl e fechar o processo
mapdl.exit()
mapdl.quit()


MapdlExitedError: MAPDL has exited